In [24]:

import torch
from torch.backends import cudnn
from torch import nn
import torchvision  # 图片、视频处理
from torchvision.transforms import ToTensor
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split    

device = 'cuda' if torch.cuda.is_available else 'cpu'
torch.__version__, torchvision.__version__, device

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
# %matplotlib inline
# vTest = 1000

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

# device = 'cpu'
print( device )


cuda


ToTensor: 
- 1、输入转为Tensor，
- 2、图片格式为 channel, height, width; 
- 3、像素取值范围规范到0/1

torch.utils.data.DataLoader, 作用：
- 乱序， shuffle 默认为 True
- 将数据采样为小批次， batch_size. batch太小会导致loss的剧烈震荡，太大则内存放不下，也会跨度过大，失去准确性；
- num_workers, 子进程设置，更多进程参与
- 设置批次处理函数 collate_fn, 用在文本等


In [25]:
# 读取数据 
sql="SELECT * FROM ds_qf60"
df = pd.read_sql_query(sql, conn, index_col=None)
del df['index']
len(df), df.head(1)

(165293,
    pct_change      diff       dea       bar  jx_days_ud60 jx_xl_250 jx_xl_120  \
 0     -1.1538 -0.108607 -0.190289  0.163366            32      None      None   
 
   jx_xl_60 jx_xl_20 jx_xl_10  ... jx_dg_250  jx_dg_30  jx_dg_20  jx_dg_10  \
 0     None     None     None  ...      None      None   3.38708   9.02793   
 
    jx_dg_5 pct_fl_1_3 pct_fl_3_10 pct_fl_5_20 pct_fl_5_60  pct_fl_20_60  
 0  33.1082   -31.6675     104.642     58.8772        None      -20.2301  
 
 [1 rows x 42 columns])

In [26]:
""" 确定训练的目标 """
df['class0'] = df['sz_jt_20']  # 10天内的阶梯涨幅

In [27]:
# 查看 class0 是否准确、百分比
df.class0.unique(), df.class0.value_counts(), df.class0.value_counts()/len(df)

(array([1., 0., 2.]),
 1.0    95821
 2.0    46832
 0.0    22640
 Name: class0, dtype: int64,
 1.0    0.579704
 2.0    0.283327
 0.0    0.136969
 Name: class0, dtype: float64)

In [28]:
#  'diff', 'dea',, 
# X_columns0 = ['pct_change', 
#              'jx_days_ud60', 
#              'jx_xl_120','jx_xl_60', 'jx_xl_20', 
#              'jx_zs_20', 
#              'lj_fl_5_20','lj_fl_20_60']
X_columns = ['jx_dg_120','jx_dg_60', 'jx_dg_20', 
             'pct_change', 
             'pct_fl_3_10','pct_fl_5_20','pct_fl_20_60']
df[ X_columns ]

,jx_dg_120,jx_dg_60,jx_dg_20,pct_change,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60
0,8.16319,-15.010000,3.387080,-1.1538,104.64200,58.8772,-20.23010
1,7.00177,-14.688300,9.715350,0.5414,155.15700,57.5850,-21.35470
2,6.34968,-14.228100,4.073800,1.8913,147.90200,37.5448,-24.00660
3,-10.72400,6.763570,0.599788,4.3909,116.82900,76.0266,3.86983
4,-1.07083,-12.851900,0.507476,0.1154,79.55220,23.4110,25.59640
...,...,...,...,...,...,...,...
165288,6.71357,-11.829100,0.265849,-3.6251,214.12400,48.4670,-3.62789
165289,10.40010,-7.082810,7.178250,4.1436,160.64500,39.3249,-9.88584
165290,-19.30980,-0.434874,1.002780,0.0000,-21.11130,23.8052,-52.85400
165291,-17.96200,1.521940,6.686400,0.9281,-5.23023,22.4991,-53.25980


In [29]:
X_data = df[X_columns]  
X = torch.from_numpy(X_data.values).type(torch.float32)
# Y_data = df.class0.values.reshape(-1,0)  # 转换成 pd type array
Y_data = df.class0.values  # 转换成 pd type array
Y = torch.from_numpy(Y_data).long()
# Y = torch.from_numpy(Y_data).type(torch.float32)
# X_data.shape, X_data.head(3)
X.shape, Y.shape

(torch.Size([165293, 7]), torch.Size([165293]))

In [30]:
# 最后检测Y_data数据准确性
unique, counts = np.unique(Y_data, return_counts=True)
unique, counts


(array([0., 1., 2.]), array([22640, 95821, 46832], dtype=int64))

In [31]:
# X,Y 形成main dataset, train/test dataset 
X = X.to(device)
Y = Y.to(device)
main_dataset = TensorDataset(X, Y)
train_ds, test_ds = random_split(main_dataset, [0.8, 0.2]) 

In [32]:
# train_ds = torchvision.datasets.MNIST('data', 
#                                       train=True, 
#                                       download=True, 
#                                       transform=ToTensor())
# test_ds = torchvision.datasets.MNIST('data', 
#                                      train=False , 
#                                      download=True, 
#                                      transform=ToTensor())
# # test_ds = test_ds.to(device)

In [47]:
# batch_size = 4
batch_size = 16
# batch_size = 64
# batch_size = 4096
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)  # batch_size开始为64,可能是最佳，或128
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [48]:
imgs, labels = next(iter(train_dl))   # train dataloder 可以分解出X和Ydata
imgs.shape, labels.shape   # 1: 黑白图片， 28x28的图片分辨率， 64张


(torch.Size([16, 7]), torch.Size([16]))

In [49]:
# imgs[0].shape, imgs[0, 0, 0]
# imgs

In [50]:
labels

tensor([1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1], device='cuda:0')

In [51]:
# 各种激活函数测试
# f(x) = max(x, 0)
# input = torch.randint(2,[2,3,4])
# input = torch.randn(5)
# input
# input, torch.relu(input)

# input,torch.sigmoid( input )
# input,torch.tanh( input )
# input,nn.LeakyReLU( input )

上面是数据准备和模式测试， 
以下正式开始设计：

In [52]:
# 多层感知器模型， 添加1个隐藏层，
class Model3( nn.Module ):   # 从nn类初始化
    def __init__(self):
        super().__init__()  # 初始化父类属性
        self.linear_1 = nn.Linear(7, 120)  # 全连接层、线性层，； 要求输入数据是1维的，所以[1, 28, 28]被压缩到1层28*28；view()
        self.linear_2 = nn.Linear(120, 84)  # 线性层， 
        self.linear_3 = nn.Linear(84, 3)  # 线性层， 
    def forward(self, input):
        x = input.view(-1, 7)
        x = torch.relu(self.linear_1(x))
        x = torch.relu(self.linear_2(x))
        logits = self.linear_3(x)            # 第3层不做激活函数， 算出在10个分量上，可能值最大的分量，就定义为分量的标签值。即哪个标签分量值最大，就是那个数字。
        return logits                         # logits: 一般指未激活前的输出2

# class Model( nn.Module ):   # 从nn类初始化
#     def __init__(self):
#         super().__init__()  # 初始化父类属性
#         self.linear_1 = nn.Linear(7, 64)  # 全连接层、线性层，； 要求输入数据是1维的，所以[1, 28, 28]被压缩到1层28*28；view()
#         self.linear_2 = nn.Linear(64, 64)  # 线性层， 
#         self.linear_3 = nn.Linear(64, 32)  # 线性层， 
#         self.linear_4 = nn.Linear(32, 4)  # 线性层， 
#     def forward(self, input):
#         x = input.view(-1, 7)
#         x = torch.relu(self.linear_1(x))
#         x = torch.relu(self.linear_2(x))
#         x = torch.relu(self.linear_3(x))
#         logits = self.linear_4(x)            # 第3层不做激活函数， 算出在10个分量上，可能值最大的分量，就定义为分量的标签值。即哪个标签分量值最大，就是那个数字。
#         return logits                         # logits: 一般指未激活前的输出2

In [53]:
model = Model().to(device)  

In [54]:
"""
- target： 0,1,2,3.。。 并不是一个独热编码的形式？
- input： logits
- output：？
"""
loss_fn = nn.CrossEntropyLoss()  
# loss_fn = nn.NLLLoss()  
# input = torch.randn(5)
# input, np.argmax(input)

In [55]:
"""
优化： 根据计算得到的损失，调整模型参数， 降低损失的过程；
- Adam 优化器
- SGD：优化model的参数、以及lr
"""
opt = torch.optim.SGD(model.parameters(), lr=0.001)

In [56]:
"""
训练循环：计算准确率correect， 以及每个批次的平均loss
"""
def train(dl, model, loss_fn, optimizer):
    size = len( dl.dataset )    # 数据多少？
    num_batches = len(dl)       # 返回训练批次
    train_loss, correct = 0, 0 # 每个批次累计的loss之和， 正确的样本数累计
    for x, y in dl:
        # x, y = x.to(device), y.to(device)
        pred = model(x)
        # pred = pred.to(torch.FloatTensor)
        loss = loss_fn(pred, y)     # 按照损失函数，计算损失
        optimizer.zero_grad()       # 清零后计算新的loop的梯度；
        loss.backward()             # 按照loss，反向计算梯度；
        optimizer.step()
        with torch.no_grad():
            # pred是2维，0维是batch数，1维才是0-9的logits输出；boll转换为float32，累计;item转换到python
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()  
            train_loss += loss.item()
    correct /= size
    train_loss /= num_batches      # losss是以每个批次计算，correct是以个数计算；所以分母不同；
    return correct, train_loss

In [57]:
# pred = model(x)

In [58]:
# 测试函数: 计算准确率correect， 以及每个批次的平均loss
def test(test_dl, model, loss_fn):
    size = len( test_dl.dataset )    # 数据多少？
    num_batches = len(test_dl)       # 返回训练批次
    test_loss, correct = 0, 0 # 每个批次累计的loss之和， 正确的样本数累计
    with torch.no_grad():
        for x, y in test_dl:
            # x, y = x.to(device), y.to(device)
            pred = model(x)
            # pred = pred.to(torch.FloatTensor)
            loss = loss_fn(pred, y)     # 按照损失函数，计算损失
            test_loss += loss.item()
            # pred是2维，0维是batch数，1维才是0-9的logits输出；boll转换为float32，累计;item转换到python
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()  
        correct /= size
        test_loss /= num_batches      # losss是以每个批次计算，correct是以个数计算；所以分母不同；
        return correct, test_loss

In [59]:
""" 
    训练 N 个 epoch， 记录每个epoch的train和test的损失、准确率。 
"""
# batch_size 非常关键：4096不准确； 64很准确。16也比较差。 ！！！
def fit(epochs, train_dl, test_dl, model, loss_fn, opt ):
    # epochs = 20
    train_loss, train_acc = [], []
    test_loss,  test_acc  = [], []

    for epoch in range( epochs ):
        epoch_acc, epoch_loss = train( train_dl, model, loss_fn, opt)
        epoch_test_acc, epoch_test_loss = test( test_dl, model, loss_fn)
        train_acc.append(epoch_acc)
        train_loss.append(epoch_loss)
        test_acc.append(epoch_test_acc)
        test_loss.append(epoch_test_loss)  # 记录、图表化后，观察是否会过拟合等问题
        
        template = ("epoch:{:2d}, train_Loss:{:.5f}, train_acc:{:.2f}, test_Loss:{:.5f}, test_acc:{:.2f}, ")
        print(template.format( epoch, epoch_loss, epoch_acc*100, epoch_test_loss, epoch_test_acc*100))
    print('Done')
    return train_loss, train_acc, test_loss, test_acc


In [60]:
# batch_size = 16
# batch_size = 64   
# batch_size = 128
# batch_size = 4
epochs = 100
# train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)  # batch_size开始为64
# test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)
train_loss, train_acc, test_loss, test_acc = fit(epochs,train_dl, test_dl, model, loss_fn, opt)


epoch: 0, train_Loss:0.96394, train_acc:57.77, test_Loss:0.95228, test_acc:57.77, 
epoch: 1, train_Loss:0.94823, train_acc:58.19, test_Loss:0.94753, test_acc:57.86, 
epoch: 2, train_Loss:0.94339, train_acc:58.27, test_Loss:0.94543, test_acc:57.85, 
epoch: 3, train_Loss:0.94017, train_acc:58.29, test_Loss:0.94745, test_acc:57.91, 
epoch: 4, train_Loss:0.93833, train_acc:58.30, test_Loss:0.94483, test_acc:57.74, 
epoch: 5, train_Loss:0.93681, train_acc:58.28, test_Loss:0.94436, test_acc:57.74, 
epoch: 6, train_Loss:0.93550, train_acc:58.37, test_Loss:0.93947, test_acc:57.90, 
epoch: 7, train_Loss:0.93460, train_acc:58.36, test_Loss:0.94351, test_acc:57.89, 
epoch: 8, train_Loss:0.93408, train_acc:58.38, test_Loss:0.94062, test_acc:58.01, 
epoch: 9, train_Loss:0.93321, train_acc:58.39, test_Loss:0.93916, test_acc:57.91, 
epoch:10, train_Loss:0.93285, train_acc:58.38, test_Loss:0.93797, test_acc:57.96, 
epoch:11, train_Loss:0.93243, train_acc:58.42, test_Loss:0.93858, test_acc:57.86, 
epoc

KeyboardInterrupt: 

In [ ]:
plt.plot( range(epochs), train_loss, label='train_loss')
plt.plot( range(epochs), test_loss, label='test_loss')
plt.legend()
# train_acc

In [ ]:
plt.plot( range(epochs), train_acc, label='train_acc')
plt.plot( range(epochs), test_acc, label='test_acc')
plt.legend()